# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

D:\Data Engineering\Data-Modeling-with-Cassandra


In [3]:
file_path_list

['D:\\Data Engineering\\Data-Modeling-with-Cassandra/event_data\\2018-11-01-events.csv',
 'D:\\Data Engineering\\Data-Modeling-with-Cassandra/event_data\\2018-11-02-events.csv',
 'D:\\Data Engineering\\Data-Modeling-with-Cassandra/event_data\\2018-11-03-events.csv',
 'D:\\Data Engineering\\Data-Modeling-with-Cassandra/event_data\\2018-11-04-events.csv',
 'D:\\Data Engineering\\Data-Modeling-with-Cassandra/event_data\\2018-11-05-events.csv',
 'D:\\Data Engineering\\Data-Modeling-with-Cassandra/event_data\\2018-11-06-events.csv',
 'D:\\Data Engineering\\Data-Modeling-with-Cassandra/event_data\\2018-11-07-events.csv',
 'D:\\Data Engineering\\Data-Modeling-with-Cassandra/event_data\\2018-11-08-events.csv',
 'D:\\Data Engineering\\Data-Modeling-with-Cassandra/event_data\\2018-11-09-events.csv',
 'D:\\Data Engineering\\Data-Modeling-with-Cassandra/event_data\\2018-11-10-events.csv',
 'D:\\Data Engineering\\Data-Modeling-with-Cassandra/event_data\\2018-11-11-events.csv',
 'D:\\Data Engineerin

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = []

# for every filepath in the file path list
for f in file_path_list:

    # reading csv file
    with open(f, "r", encoding="utf8", newline="") as csvfile:
        # creating a csv reader object
        csvreader = csv.reader(csvfile)
        next(csvreader)

        # extracting each data row one by one and append it
        for line in csvreader:
            # print(line)
            full_data_rows_list.append(line)

# uncomment the code below if you would like to get total number of rows
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect("myDialect", quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open("event_datafile_new.csv", "w", encoding="utf8", newline="") as f:
    writer = csv.writer(f, dialect="myDialect")
    writer.writerow(
        [
            "artist",
            "firstName",
            "gender",
            "itemInSession",
            "lastName",
            "length",
            "level",
            "location",
            "sessionId",
            "song",
            "userId",
        ]
    )
    for row in full_data_rows_list:
        if row[0] == "":
            continue
        writer.writerow(
            (
                row[0],
                row[2],
                row[3],
                row[4],
                row[5],
                row[6],
                row[7],
                row[8],
                row[12],
                row[13],
                row[16],
            )
        )

In [5]:
# check the number of rows in your csv file
with open("event_datafile_new.csv", "r", encoding="utf8") as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [6]:
from cassandra.cluster import Cluster

try:
    cluster = Cluster(["127.0.0.1"])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [7]:
# Create a Keyspace
session.execute(
    """
    CREATE KEYSPACE IF NOT EXISTS modeling
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """
);

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('modeling')
except Exception as e:
    print(e)

## Create queries

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession   = 4

#### The table will containt those columns:
- **sessionId**, **itemInSession** as _partition keys_ for filtering and gurantee uniqueness.
- clustering columns
    - **artist**
    - **song_name**
    - **song_length**

In [9]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1 = "SELECT artist, song_title, song_length FROM songs_by_user_session WHERE session_id = 338 AND item_in_session = 4"

In [10]:
session.execute(
    """
    DROP TABLE IF EXISTS songs_by_user_session
    """
);
session.execute(
    """
    CREATE TABLE IF NOT EXISTS songs_by_user_session
    (session_id int, item_in_session int, artist varchar, song_title varchar,
    song_length float,  PRIMARY KEY(session_id, item_in_session))
    ;"""
);

In [11]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'
lines = []
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        lines.append(line)

In [12]:
file = 'event_datafile_new.csv'

In [13]:
lines[6]

['Girl Talk',
 'Kaylee',
 'F',
 '8',
 'Summers',
 '160.15628',
 'free',
 'Phoenix-Mesa-Scottsdale, AZ',
 '139',
 'Once again',
 '8']

In [14]:
import pandas as pd
for i, c in enumerate(pd.read_csv(file).columns):
    print(c, i)

artist 0
firstName 1
gender 2
itemInSession 3
lastName 4
length 5
level 6
location 7
sessionId 8
song 9
userId 10


In [15]:
df = pd.read_csv(file)

In [16]:
df[df.sessionId == 388]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
1236,Pulp,Matthew,M,1,Jones,244.08771,free,"Janesville-Beloit, WI",388,Babies,36


In [17]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = "event_datafile_new.csv"

with open(file, encoding="utf8") as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_user_session (session_id, item_in_session, artist, song_title, song_length) "
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(
            query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]))
        )

#### Do a SELECT to verify that the data have been inserted into each table

In [18]:
rows = session.execute(query1)
for x in rows:
    print(x)

Row(artist='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


### 2. Give me only the following: name of artist, song (sorted by itemInSession**) and user (first and last name) for userid = 10, sessionid = 182
#### The table will containt those columns:
- **sessionId**, **userid** (to filter by )also **itemInSession** _composite partition keys_ for sorting.
- clustering columns
    - **artist**
    - **song_name**
    - **first_name**
    - **last_name**


In [19]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2 = "SELECT artist, song_title, first_name, last_name FROM artists_song_by_user_session WHERE userid = 10 AND sessionid = 182"

In [20]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3 = """SELECT first_name, last_name FROM users_by_song WHERE song_name = 'All Hands Against His Own'"""

In [21]:
session.execute(
    """
    DROP TABLE IF EXISTS artists_song_by_user_session
    """
);
session.execute(
    """
    CREATE TABLE IF NOT EXISTS artists_song_by_user_session
    (userid int, sessionid int, item_in_session int, artist varchar, song_title varchar, first_name varchar, last_name varchar,
    PRIMARY KEY ((userid, sessionid), item_in_session))
    """
);

In [22]:
import pandas as pd
for i, c in enumerate(pd.read_csv(file).columns):
    print(c, i)

artist 0
firstName 1
gender 2
itemInSession 3
lastName 4
length 5
level 6
location 7
sessionId 8
song 9
userId 10


In [23]:
with open(file, encoding="utf8") as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    for line in csvreader:
        query = "INSERT INTO artists_song_by_user_session (userid, sessionid, item_in_session ,artist, song_title, first_name, last_name) "
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(
            query,
            (
                int(line[10]),
                int(line[8]),
                int(line[3]),
                line[0],
                line[9],
                line[1],
                line[4],
            ),
        )

In [24]:
rows = session.execute(query2)
for x in rows:
    print(x)

Row(artist='Down To The Bone', song_title="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist='Three Drives', song_title='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist='Sebastien Tellier', song_title='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
- **song_name**, **userid** as _partition keys_ to filter by.
- clustering columns
    - **first_name**
    - **last_name**

In [25]:
session.execute(
    """
    DROP TABLE IF EXISTS users_by_song
    """
);
session.execute(
    """
    CREATE TABLE IF NOT EXISTS users_by_song
    (song_name varchar, user_id int, first_name varchar, last_name varchar, PRIMARY KEY(song_name, user_id))
    """
);

In [26]:
import pandas as pd
for i, c in enumerate(pd.read_csv(file).columns):
    print(c, i)

artist 0
firstName 1
gender 2
itemInSession 3
lastName 4
length 5
level 6
location 7
sessionId 8
song 9
userId 10


In [27]:
with open(file, encoding="utf8") as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    for line in csvreader:
        query = "INSERT INTO users_by_song (song_name, user_id, first_name, last_name) "
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]) ,line[1], line[4]))

In [28]:
rows = session.execute(query3)
for x in rows:
    print(x)

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')
Row(first_name='Sara', last_name='Johnson')


### Drop the tables before closing out the sessions

In [29]:
session.execute(
    """
    DROP TABLE IF EXISTS songs_by_user_session
    """
);
session.execute(
    """
    DROP TABLE IF EXISTS artists_song_by_user_session
    """
);
session.execute(
    """
    DROP TABLE IF EXISTS users_by_song
    """
);

### Close the session and cluster connection¶

In [30]:
session.shutdown()
cluster.shutdown()